# Greedy approach

### Parameters

In [ ]:
# Define your lower bound here:
zeta = -1807;        
folder = ['data/6045/']
example = ['NewExample1']

In [ ]:
import math
import numpy as np
import cplex  as CPX
import time
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import copy

In [ ]:
def SecModelParameter ( CGLP_m, n, CGLP_rhs, CGLP_Coef,Br_var):
    MultiplyVar=np.arange(Br_var+1,n).tolist()
    Colij=dict()
    k=n
    for i in MultiplyVar:
        for j in range (n):
            if i!=j:
                if (tuple([j,i]) in Colij)!=True:
                    if i<j:
                        Colij[tuple([i,j])]= k
                        k=k+1
                    else:
                        Colij[tuple([j,i])]= k
                        k=k+1
    lenColij=len(Colij)+n
    CompleteMatrix=[]
    CompleteRHS=[]
    for i in MultiplyVar:
        for t in range (CGLP_m):
            LineMatrix=np.zeros(lenColij)
            for j in range (n):
                if i==j:
                    LineMatrix[j]= CGLP_Coef[t][j]
                    LineMatrix[j]=LineMatrix[j]-CGLP_rhs[t]
                    
                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=CGLP_Coef[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=CGLP_Coef[t][j]    
            CompleteMatrix.append(LineMatrix)
            CompleteRHS.append(0)



            LineMatrix=np.zeros(lenColij)
            for j in range (n):
                if j==i:
                    LineMatrix[i]= CGLP_rhs[t]
                else:
                    LineMatrix[j]= CGLP_Coef[t][j]
                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=-1*CGLP_Coef[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=-1*CGLP_Coef[t][j]   
            CompleteMatrix.append(LineMatrix)   
            CompleteRHS.append(CGLP_rhs[t])
        
        

    for i in MultiplyVar:
        for t in range (2*n):
            LineMatrix=np.zeros(lenColij)
            LineRHS=np.zeros(1)
            if my_rhsUnit[t]==1:
                for j in range (n):
                    LineRHS= my_rhsUnit[t]
                    if i==j:
                        LineMatrix[j]= UnitMatrix[t][j]
                        LineMatrix[j]=LineMatrix[j]-my_rhsUnit[t] 
                    if i<j:

                        LineMatrix [Colij[tuple([i,j])]]=UnitMatrix[t][j]
                    if i>j:

                        LineMatrix [Colij[tuple([j,i])]]=UnitMatrix[t][j] 

            if all(k == 0 for k in LineMatrix)!=True:
                CompleteMatrix.append(LineMatrix) 
                CompleteRHS.append(0)

            LineMatrix=np.zeros(lenColij)
            LineRHS=np.zeros(1)
            for j in range (n):
                LineRHS= my_rhsUnit[t]
                if j==i:
                    LineMatrix[i]= my_rhsUnit[t]
                else:
                    LineMatrix[j]=UnitMatrix[t][j]

                if i<j:
                    LineMatrix [Colij[tuple([i,j])]]=-1*UnitMatrix[t][j]
                if i>j:
                    LineMatrix [Colij[tuple([j,i])]]=-1*UnitMatrix[t][j]   

            if all(k == 0 for k in LineMatrix)!=True:
                CompleteMatrix.append(LineMatrix) 
                CompleteRHS.append(LineRHS)

                    
    return CompleteMatrix,CompleteRHS

In [ ]:
def BB (m,n,A, my_obj,b, lx, ux, IntVars,my_colnames, zeta):
    lp_solving=0
    eps=0.001
    extraeps=0.01
    cLR = 0.06
    Threshold = 0.6
    branch_flag=-1
    prev_Br_var=-2
    Br_var=-1
    node_numbers=0
    Tol = 0.0000001;
    LXS = np.array([np.copy(lx)]);
    UXS = np.array([np.copy(ux)]);
    ZS = np.array([100000]);
    Br_order=np.array([-1])
    x = [];
    prev_zeta= zeta 
    nn = 1;
    while 1==1:
        f=list(np.where(ZS <= zeta + Tol)[0])
        if len(f)>0:
            LXS=np.delete(LXS, f, 0)
            UXS=np.delete(UXS, f, 0)
            ZS=np.delete(ZS, f)
            Br_order=np.delete(Br_order,f)
            nn = nn - len(f);
        if nn==0:
            break
        lx1=LXS[0]
        ux1=UXS[0] 
        LXS=np.delete(LXS, 0, 0)
        UXS=np.delete(UXS, 0, 0)
        ZS=np.delete(ZS, 0)
        prev_Br_var=Br_var
        Br_var=Br_order[0]
        Br_order=np.delete(Br_order,0)
        nn = nn - 1;
        node_numbers=node_numbers+1
        Lp_Relaxation = CPX.Cplex() 
        Lp_Relaxation.objective.set_sense(Lp_Relaxation.objective.sense.maximize)    
        Lp_Relaxation.variables.add(obj = my_obj, lb=lx1, ub =ux1, names = my_colnames)
        Lp_Relaxation.linear_constraints.add(lin_expr = my_coef, senses = 'L'*m, rhs = my_rhs) 
        Lp_Relaxation.parameters.simplex.display.set(0)
        Lp_Relaxation.solve()

        status= Lp_Relaxation.solution.get_status()
        if status != 3:
            xR= Lp_Relaxation.solution.get_values()
            zR= Lp_Relaxation.solution.get_objective_value()
        if status == 2:
            check_unbound=BB (m,n,A, np.zeros[n],b, lx, ux, IntVars,my_colnames)
            zR0= check_unbound[0]
            if zR0==0:
                zeta = math.inf
                break
        if (status == 3 or zR<=zeta + Tol):
            continue 

        Fractions=np.abs(xR - np.round(xR))
        maxf=max(Fractions)
        if maxf <= Tol:     
            x = xR;
            zeta = zR;
            continue;  

    #####################################################################################################               
        VarsValue=np.copy(lx1[0:Br_var+1])

        if Br_var not in [-1,n-1]:
            if Br_var>branch_flag or prev_zeta!=zeta: 

                if prev_zeta!=zeta:
                    prev_zeta=zeta
                    CGLP_rhs[-1]=-1*zeta

                SecModelParameter_output=SecModelParameter(CGLP_m, n, CGLP_rhs, CGLP_Coef,Br_var)
                CompleteMatrix=SecModelParameter_output[0]
                CompleteRHS=SecModelParameter_output[1]
                numrows = len(CompleteMatrix)    
                numcols = len(CompleteMatrix[0])
                my_lb=[0.0]*numrows
                my_row=[1.0]*numrows

                Fliped_CompleteMatrix_const=np.negative(np.array(CompleteMatrix)).T.tolist()

                #####train
                if Br_var>branch_flag:
                    FinalCoeff=Fliped_CompleteMatrix_const[Br_var+1:]
                    FinalBound=np.negative(np.eye(numrows, dtype=int)).tolist()
                    FinalCoeff=FinalCoeff+FinalBound
                    lenFinalCoeff=len(FinalCoeff)
                    FinalCoeff=np.array(FinalCoeff)

                    # negative of regular const class one
                    X_trainset=np.negative(FinalCoeff)                    
                    #one vector class one
                    OnesVector=np.ones((lenFinalCoeff,numrows))
                    newrow=FinalCoeff+OnesVector*eps 
                    X_trainset=np.concatenate((X_trainset,newrow), axis=0)
                    # class zero
                    for k in range(2):
                        PosRandNum=np.random.uniform(1,10000,(lenFinalCoeff,numrows))  
                        newrow=FinalCoeff-PosRandNum*extraeps 
                        X_trainset=np.concatenate((X_trainset,newrow), axis=0)
                        
                    X_trainset=preprocessing.normalize(X_trainset, axis=1, norm='l2')
                    y_trainset=[1]*lenFinalCoeff*2+ [0]*lenFinalCoeff*2
                    LR = LogisticRegression(C=cLR, solver= 'liblinear').fit(X_trainset,y_trainset)
#                     cLR=cLR+0.005
                branch_flag=Br_var

                obj_mat=np.array(CompleteMatrix)[:,range(0,Br_var+1)]

            obj=[]
            for j in range (numrows):
                obj.append(np.inner(obj_mat[j],VarsValue)-CompleteRHS[j])
            X_testset=[obj/np.linalg.norm(obj)]
            if (LR.predict_proba(X_testset)[:, 1] >= Threshold).astype(int):  
                continue

    #####################################################################################################                
        indf=Br_var+1
        nlx = np.array(np.copy(lx1));
        nlx[indf] = 1;    
        nux = np.array(np.copy(ux1));
        nux[indf] = 0;
        LXS=np.append(LXS,(lx1, nlx),axis=0)
        UXS=np.append(UXS,(nux, ux1),axis=0)
        ZS = np.append(ZS, (zR, zR));
        Br_order=np.append(Br_order,(indf,indf))
        nn = nn + 2    
    return zeta,node_numbers,lp_solving  

In [ ]:
for xx in ['python3/6045/']:
    for yy in ['NewExample31']:
        main_prob = CPX.Cplex(xx+yy+'.lp')
        m=main_prob.linear_constraints.get_num()
        CGLP_m=m+1
        n=main_prob.variables.get_num()
        my_rhs=np.array([])
        CGLP_rhs=np.array([])
        my_coef=[]
        CGLP_Coef=[]

        for i in range(m):
            temp=[]
            for j in range(n):
                temp.append (main_prob.linear_constraints.get_coefficients(i,j))    
            my_coef.append([list (range(n)),temp])
            CGLP_Coef.append(temp)
        
        CGLP_Coef = np.vstack((CGLP_Coef, main_prob.objective.get_linear()))
        CGLP_Coef[-1]=np.negative(CGLP_Coef[-1])
        for i in range (m):
            my_rhs=np.append(my_rhs,main_prob.linear_constraints.get_rhs(i))
            CGLP_rhs=np.append(CGLP_rhs,main_prob.linear_constraints.get_rhs(i))
        CGLP_rhs=np.append(CGLP_rhs,0)
        my_obj= np.array(main_prob.objective.get_linear())
        UnitMatrix=np.eye(n, dtype=int).tolist()+np.negative(np.eye(n, dtype=int)).tolist()
        my_rhsUnit=[1]*n+[0]*n 
        IntVars=np.arange(n)
        lx= [0.0]*n
        ux=[1.0]*n
        my_colnames=[]
        for i in range(n):
            my_colnames.append("x"+str(i))  

        time1=time.time()
        output=BB (m,n,my_coef,my_obj,my_rhs, lx, ux, IntVars,my_colnames,zeta)               
        time2=time.time()-time1

        with open(xx+yy+'bb_greedy_optimality.txt', 'a') as f:
            f.write('{}:{}\n'.format('objective value',output[0]))
            f.write('{}:{}\n'.format('Number of explored node',output[1]))
            f.write('{}:{}\n'.format('Number of solved LPs',output[2]))
            f.write('{}:{}\n'.format('ElapsedTime',time2))
        f.closed